In [ ]:
import sys, time, os, asyncio, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient
from lsst.summit.utils.utils import dayObsIntToString
from lsst.summit.utils.efdUtils import calcNextDay


In [ ]:
client = EfdClient('summit_efd')

In [ ]:
pdf = PdfPages("/home/cslage/DATA/Shutter_State_21Aug25.pdf")
startDay = 20250310
endDay = 20250326
fig = plt.figure(figsize=(15,5))
dayObs = startDay
while dayObs <= endDay:
    nextDayObs = calcNextDay(dayObs)
    dayString = dayObsIntToString(dayObs)
    nextDayString = dayObsIntToString(nextDayObs)
    
    try:
        start = Time(f"{dayString}T18:00:00", scale='utc')
        end = Time(f"{nextDayString}T14:00:00", scale='utc')
    
        state = await client.select_time_series('lsst.sal.ATDome.logevent_mainDoorState', \
                                            ['*'],  start, end)
        shut = await client.select_time_series('lsst.sal.ATDome.position', \
                                                    ['mainDoorOpeningPercentage'],  start, end)
        close_command = await client.select_time_series('lsst.sal.ATDome.command_closeShutter', \
                                                    ['*'],  start, end)
        open_command = await client.select_time_series('lsst.sal.ATDome.command_openShutter', \
                                                    ['*'],  start, end)
        summary_state = await client.select_time_series('lsst.sal.ATDome.logevent_summaryState', \
                                                    ['*'],  start, end)
    
        dome_move = await client.select_time_series('lsst.sal.ATDome.command_moveAzimuth', \
                                                    ['*'],  start, end)
        comm = await client.select_time_series('lsst.sal.ATDome.logevent_scbLink', \
                                                    ['active'],  start, end)
        if len(comm) == 0:
            comm_loss = pd.DataFrame()
    
        else:
            comm_loss = comm[comm['active']==False]
        
        print(dayObs, len(state), len(shut), len(close_command), len(summary_state), len(dome_move))
        if len(shut) == 0:
            print(f"{dayObs} has no shuts")
            dayObs = calcNextDay(dayObs)
            continue
        plt.clf()
        ax = fig.subplots(1,1)
        ax.set_title(f"Shutter_State_{dayObs}", fontsize=24)
        #state['state'].plot(ax=ax, color='blue', label='Actual State')
        states = state['state'].values
        tstates = tstates = state.index
        ax.step(tstates, states, color='blue', label='Actual State', where='post')        
        
        ax.set_yticks([1,2,3,4,5,6,7])
    
        ax.set_yticklabels(['Closed', 'Opened', 'Ajar', 'Opening', 'Closing','',''])
        ax.set_ylabel("Door State")
        ax1 = ax.twinx()
        shut['mainDoorOpeningPercentage'].plot(ax=ax1, color='red', label='Shutter Open Percentage')
        ax1.set_ylabel("Open percentage")
        ax1.set_ylim(-2,115)
    
        disabled = summary_state[summary_state['summaryState'] == 1]
        enabled = summary_state[summary_state['summaryState'] == 2]
        standby = summary_state[summary_state['summaryState'] == 5]
        fault = summary_state[summary_state['summaryState'] == 3]
    
        """
        disabled = Time("2024-02-12T12:00:00", scale='utc')
        enabled = Time("2024-02-12T18:44:00", scale='utc')
        ax1.axvline(disabled.isot, ls='--', color='cyan', label='DISABLED')
        ax1.axvline(enabled.isot, ls='--', color='orange', label='ENABLED')
        """
        for close in close_command.index[0:1]:
            ax1.axvline(close, ls='-.', color='green', label='Close Command')
        for open in open_command.index[0:1]:
            ax1.axvline(open, ls='-.', color='magenta', label='Open Command')
        for disable in disabled.index[0:1]:
            ax1.axvline(disable, ls='--', color='cyan', label='Disabled')
        for enable in enabled.index[0:1]:
            ax1.axvline(enable, ls='--', color='darkgreen', label='Enabled')
        for std in standby.index[0:1]:
            ax1.axvline(std, ls='--', color='orange', label='Standby')
        for flt in fault.index[0:1]:
            ax1.axvline(flt, ls='--', color='red', label='Fault')
        for move in dome_move.index[0:1]:
            ax1.axvline(move, ymin=0, ymax=0.1, ls='--', color='black', label='Dome_Move')
        for loss in comm_loss.index[0:1]:
            ax1.axvline(loss, ls='--', color='lime', label='Main/Top Comm Loss')
        """    
        for i, this_enable in enumerate(enables):
            for en in this_enable.index[0:1]:
                print(cscs[i], en)
                ax1.axvline(en, ls='--',  label=cscs[i])
        
        for i, this_enable in enumerate(enables):
            for en in this_enable.index:
                print(cscs[i], en)
                ax1.axvline(en, ls='--',  label='')
        """        
    
        for close in close_command.index:
            ax1.axvline(close, ls='-.', color='green', label='')
        for open in open_command.index:
            ax1.axvline(open, ls='-.', color='magenta', label='')
        for disable in disabled.index:
            ax1.axvline(disable, ls='--', color='cyan', label='')
        for enable in enabled.index:
            ax1.axvline(enable, ls='--', color='darkgreen', label='')
        for std in standby.index:
            ax1.axvline(std, ls='--', color='orange', label='')
        for flt in fault.index:
            ax1.axvline(flt, ls='--', color='red', label='')
        for move in dome_move.index:
            ax1.axvline(move, ymin=0, ymax=0.1, ls='--', color='black', label='')
        for loss in comm_loss.index:
            ax1.axvline(loss, ls='--', color='lime', label='')
    
    
        ax.legend(loc='lower left')
        ax1.legend(loc='upper left')
        #startPlot = pd.Timestamp("2024-07-16T22:00:00", tz='UTC')
        #endPlot = pd.Timestamp("2024-07-17T02:30:00", tz='UTC')
        #ax.set_xlim(startPlot, endPlot)
        
        pdf.savefig(fig)  # saves the current figure into a pdf page
        plt.clf()
        dayObs = calcNextDay(dayObs)
    except:
        print(f"{dayObs} plot failed")
        dayObs = calcNextDay(dayObs)
        continue
pdf.close()


In [ ]:
%matplotlib inline
summary_state['summaryState'].plot()


In [ ]:
summary_state['summaryState']